In [1]:
#Importing the necessary libraries:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import numpy as np
from datetime import datetime, timedelta
import json
import random

In [ ]:
#Creating an empty list that will store the book information:
book_info = []
count = 0
ratings = []

#Iterating over 2000 books on Goodreads:
for i in range(1,2000):
  url = f'https://www.goodreads.com/book/show/{i}'
  page = urlopen(url)

  html_bytes = page.read()
  html = html_bytes.decode('utf-8')

  my_soup = BeautifulSoup(html,'html.parser')

  #Obtaining information on the book such as name, author, rating:
  content = my_soup.find_all("script", type = "application/ld+json")

  #Obtaining information on the books genres:
  genres =  my_soup.find_all("span", class_= "BookPageMetadataSection__genreButton")
  genres = [g.getText() for g in genres]

  #Obtaining the book reviews information:
  reviews = my_soup.find_all("section",class_ = "ReviewText__content")
  review_date = my_soup.find_all("section",class_ = "ReviewCard__row")
  reviewers = my_soup.find_all("div",class_ = "ReviewerProfile__name")
  ratings = my_soup.find_all("div",class_ = "ShelfStatus")

  reviews = [r.getText() for r in reviews]
  review_date = [rd.getText() for rd in review_date]
  reviewers = [rv.getText() for rv in reviewers]

  #Taking a break every 200 books to avoid getting blocked by the website:
  if count == 200:
    time.sleep(6)
    count = 0

  if len(content)== 0:
    continue
  book_info += [[content[0].getText(),genres,reviews,review_date,reviewers,ratings]]

  #Increasing our count:
  count += 1
  print (f'Book number: {i}')

#Observing how many books we have scraped:
print (f'\nNo. of books scraped: {len(book_info)}.')

Book number: 1
Book number: 2
Book number: 3
Book number: 4
Book number: 5
Book number: 6
Book number: 7
Book number: 8
Book number: 9
Book number: 10
Book number: 11
Book number: 12
Book number: 13
Book number: 14
Book number: 15
Book number: 16
Book number: 17
Book number: 18
Book number: 19
Book number: 21
Book number: 22
Book number: 23
Book number: 24
Book number: 25
Book number: 26
Book number: 27
Book number: 28
Book number: 29
Book number: 30
Book number: 31
Book number: 32
Book number: 33
Book number: 34
Book number: 35
Book number: 36
Book number: 37
Book number: 38
Book number: 39
Book number: 40
Book number: 45
Book number: 46
Book number: 48
Book number: 49
Book number: 50
Book number: 51
Book number: 52
Book number: 53
Book number: 54
Book number: 55
Book number: 56
Book number: 57
Book number: 58
Book number: 59
Book number: 60
Book number: 61
Book number: 63
Book number: 65
Book number: 66
Book number: 67
Book number: 68
Book number: 69
Book number: 71
Book number: 72
B

In [ ]:
#Saving our content to a data-frame for later use:
original_content = [b for b in book_info]
original_df = pd.DataFrame({'Content':original_content})
original_df.to_csv('original.csv')

# **REVIEWS:**

Obtaining the books review information from Goodreads. Here, we obtain;
1. Reviewer
2. ReviewDate
3. Rating

We then store this information in .csv files, a reviewers file and a file where we store the reviews and to which book they belong.

In [ ]:
#Creating a list that will contain the reviews:
book_reviews = list()

#Iterating across all books:
for book_ in book_info:
  r_list = book_[2]
  #Making sure the book has reviews before continuing:
  if r_list:
    book_reviews += [r for r in r_list]

print (f'No. of reviews: {len(book_reviews)}')

No. of reviews: 33202


In [ ]:
#Creating a list that contains all review dates:
book_review_dates = list()

#Iterating across all the books and checking the validity of the dates:
for book_ in book_info:
  rd_list = book_[3]
  #Checking to make sure the review has a date:
  if rd_list:
    for rd in rd_list:
      #Splitting the list to
      split_rd = re.split(r'(?=[A-Z])',rd)
      for dd in split_rd:
        try:
          res = bool(datetime.strptime(dd,"%B %d, %Y"))
          if res:
            book_review_dates += [datetime.strptime(dd,"%B %d, %Y").date()]
        except ValueError:
          res = False

print (f'No. of review dates: {len(book_review_dates)}')

No. of review dates: 33202


In [ ]:
#Creating a list that contains all the reviewers:
book_reviewers = list()

#Iterating across all the books:
for book_ in book_info:
  rv_list = book_[4]
  if rv_list:
    book_reviewers += [rv for rv in rv_list]

print (f'No. of reviewers: {len(book_reviewers)}')

No. of reviewers: 33202


In [ ]:
#Creating a list that contains all the ratings:
book_ratings = list()

#Iterating acorss all the books:
for book_ in book_info:
  rt_list = book_[5]
  if rt_list:
    for rt in rt_list:
      rt_child = rt.find("span", class_="RatingStars RatingStars__small")
      #Making sure the tag exists before adding the rating to the list:
      if rt_child:
        book_ratings += [rt_child["aria-label"]]
      else:
        book_ratings.append('NULL')

#Converting the strings to numbers in book_ratings:
book_ratings = [int(brt.split()[1]) if brt != "NULL" else "NULL" for brt in book_ratings]

In [ ]:
#Checking the length of each of our lists to ensure no extra or missing data:
print ("No. of book reviews:",len(book_reviews))
print ("No. of reviewers:",len(book_reviewers))
print ("No. of review dates:",len(book_review_dates))
print ("No. of book ratings:",len(book_ratings))

No. of book reviews: 33202
No. of reviewers: 33202
No. of review dates: 33202
No. of book ratings: 33202


In [ ]:
#Creating a list of book ids that matches the number of reviews:
book_id = list()

for i in range(len(book_info)):
  for j in range(len(book_info[i][2])):
    book_id += [i+1]

print (f'No. of book_ids: {len(book_id)}.')

No. of book_ids: 33202.


In [ ]:
#Creating our reviewer_id list:
reviewer_id = list(range(1,len(book_reviewers)+1))

#Creating our data-frame that contains reviewer information:
reviewer_df = {"Username":book_reviewers}
reviewer_df = pd.DataFrame(reviewer_df, index = reviewer_id)
reviewer_df

,Username
1,Jayson
2,Jayson
3,Whitney Atkinson
4,Nicole
5,Tharindu Dissanayake
...,...
33198,Olivia Cornwell
33199,Nicole
33200,Cheri Herald
33201,Susy


In [ ]:
#Saving our reviewers to a csv file for later use:
reviewer_df.to_csv('reviewers.csv')

In [ ]:
#Creating our review df:

#Creating our review id's:
review_id = list(range(1,len(book_reviews)+1))

review_df = {"ReviewerID":reviewer_id, "Review":book_reviews,
             "BookID":book_id,"Date":book_review_dates,"Rating":book_ratings}

review_df = pd.DataFrame(review_df, index = review_id)
review_df

,ReviewerID,Review,BookID,Date,Rating
1,1,(A) 86% | ExtraordinaryNotes: A relationshippy...,1,2024-07-22,5
2,2,(A) 86% | ExtraordinaryNotes: It dwells on the...,1,2022-03-13,5
3,3,i dont want to talk to anyonedont even look at...,1,2015-05-26,5
4,4,"2023: Jak co reread, to mój ulubieniec. 2021: ...",1,2023-02-20,5
5,5,"""5. Should you feel that a family member, coll...",1,2021-01-10,5
...,...,...,...,...,...
33198,33198,A nice biography. It has fun little trivia box...,1931,2013-12-17,3
33199,33199,"Simple, quick read",1931,2016-02-07,3
33200,33200,Short and sweet. Easy read-- seems intended fo...,1931,2023-11-26,4
33201,33201,Just done with this book concise and precise.....,1931,2012-02-03,4


In [ ]:
#Saving to a data-frame:
review_df.to_csv('review.csv')

# **GENRES:**

Obtaining the genre information from the books. This is done by iterating through each book and extracting their genre list; we then add each genre to the list that will store them. We also create a dictionary where each genre is stored with an ID number.

In [ ]:
#Obtaining the genres from the books:

#Creating an empty list that will contain our genres:
book_genres = list()

#Iterating through all available books and adding their genres to book_genres:
for book_ in book_info:
  g_list = book_[1]
  #Checking to see if g_list is empty; if not, we add its genres to the list:
  if g_list:
    book_genres += [g for g in g_list]
print (len(book_genres))

8463


In [ ]:
#Obtaining the unique genres to avoid repeating checks:
book_genres = list(np.unique(book_genres))
print (len(book_genres))

395


In [ ]:
#Creating a list where we will store the genre descriptions:
genre_descriptions = list()

#Obtaining the genre descriptions:
for g in book_genres:
  #Making sure we can see our current genre:
  print (f"Current genre: {g}")

  url = f"https://www.goodreads.com/genres/{g}"
  url = url.replace(" ","%20")
  if "é" in url:
    url = url.replace("é","e")
  page = urlopen(url)

  html_bytes = page.read()
  html = html_bytes.decode('utf-8')

  genre_soup = BeautifulSoup(html,'html.parser')

  genre_desc = genre_soup.find("div", class_ = "mediumText reviewText")
  #Checking that the genre has a description:
  if genre_desc:
    genre_descriptions += [genre_desc.getText().replace('\n','')]
  else:
    genre_descriptions.append("NULL")


Current genre: 14th Century
Current genre: 17th Century
Current genre: 19th Century
Current genre: 20th Century
Current genre: 2nd Grade
Current genre: Academia
Current genre: Academics
Current genre: Action
Current genre: Adoption
Current genre: Adult
Current genre: Adult Fiction
Current genre: Adventure
Current genre: Africa
Current genre: African American
Current genre: African Literature
Current genre: Alternate History
Current genre: Alternative Medicine
Current genre: Amazon
Current genre: American
Current genre: American History
Current genre: American Revolution
Current genre: Americana
Current genre: Ancient
Current genre: Ancient History
Current genre: Animals
Current genre: Anthologies
Current genre: Anthropology
Current genre: Apocalyptic
Current genre: Archaeology
Current genre: Architecture
Current genre: Art
Current genre: Art Design
Current genre: Art History
Current genre: Asia
Current genre: Asian Literature
Current genre: Astronomy
Current genre: Audiobook
Current ge

In [ ]:
#Making sure genres only appear once:
book_genres = np.unique(book_genres)
genre_id = range(1,len(book_genres)+1)

genre_info = pd.DataFrame({'Genre':book_genres, "Description":genre_descriptions},index = genre_id)

#Displaying the genres and their information:
genre_info

,Genre,Description
1,14th Century,"As a means of recording the passage of time, t..."
2,17th Century,The 17th century was the century that lasted f...
3,19th Century,Novels or collections of stories set during th...
4,20th Century,The 20th century was the period between Januar...
5,2nd Grade,Books appropriate for 2nd grade reading level.
...,...,...
391,World History,"World history, global history or transnational..."
392,Writing,Books about writing may include guidelines abo...
393,Young Adult,Young-adult fiction (often abbreviated as YA) ...
394,Zen,NULL


In [ ]:
#Placing our genre df in a csv:
genre_info.to_csv('genre.csv')

# **Authors:**

Extracting information on the authors including:
1. First Name.
2. Last Name.
3. About.

This information is then stored in a data-frame, with the afforementioned information as well as the author ID's.

In [ ]:
#Obtaining the first and last names of the Authors:

#Counting how many authors there are:
author_count = int()

#Creating lists where we will store the names of the authors and their Goodreads profiles:
author_names = list()
author_page = list()

#Iterating through the books:
for book_ in book_info:
  book_ = book_[0].split('@type')
  for row in book_:
    author_count += row.count('Person')
    if row.count('Person')>=1:
      a_split = row.translate({ord('{'):None}).translate({ord('}'):None}).split(',')
      author_names += [a_split[1]]
      if "url" not in a_split[2]:
        author_page += ["NULL"]
        continue
      author_page += [a_split[2]]

#Cleaning up the data by removing unnecessary terms:
author_names = [an.replace('"name":','') for an in author_names]

#Pulling the AboutAuthor page urls:
author_urls = list()
for ap in author_page:
  if len(ap.split(':',1))>=2:
    ap = ap.split(':',1)[1].translate({ord('"'):None})
    author_urls += [ap]
  else:
    author_urls += [None]

In [ ]:
#Creating a dictionary of authors and their pages to avoid repetition later on:
combo_name_url = dict(zip(author_names,author_urls))

print (f'No. of unique authors and urls: {len(combo_name_url)}')

No. of unique authors and urls: 1922


In [ ]:
#Making the author urls list shorter:
author_urls = list(combo_name_url.values())

In [ ]:
i = 1
for jj in combo_name_url:
  print (i,jj)
  i += 1

1 "J.K. Rowling"
2 "Mary GrandPré"
3 "W. Frederick Zimmerman"
4 "Douglas Adams"
5 "Jamie Flinchbaugh"
6 "Andy Carlino"
7 "Dennis Pawley"
8 "Stephen Fry"
9 "Bill Bryson"
10 "J.R.R. Tolkien"
11 "Francis Ledoux"
12 "Maria Skibniewska"
13 "Alan  Lee"
14 "Chris   Smith"
15 "Christopher  Lee"
16 "Richard Taylor"
17 "Jane Johnson"
18 "Jude Fisher"
19 "Brian Sibley"
20 "Michael Bakewell"
21 "James Hamilton-Paterson"
22 "Dave         Thomas"
23 "David Heinemeier Hansson"
24 "Leon Breedt"
25 "Mike   Clark"
26 "Thomas  Fuchs"
27 "Andreas Schwarz"
28 "Mark   Watson"
29 "Edith Wharton"
30 "Marion Mainwaring"
31 "Luther Butler"
32 "Gary Paulsen"
33 "Donna Ickes"
34 "Edward Sciranko"
35 "Keith Vasconcelles"
36 "Molly Hatchet"
37 "Dale Peck"
38 "Bud Cheff Sr."
39 "Angela Knight"
40 "Kate Douglas"
41 "Shelby Morgen"
42 "Kate  Hill"
43 "Sahara Kelly"
44 "Judy Mays"
45 "Marteeka Karland"
46 "Willa Okati"
47 "Lacey Savage"
48 "Delia Sherman"
49 "Patricia A. McKillip"
50 "Anne McCaffrey"
51 "Elizabeth Ann 

In [ ]:
#Removing problematic urls:
combo_name_url.pop('"paul-auster-national-public-radio-staff"')
combo_name_url.pop('"david-bach"')
combo_name_url.pop('"Kevin O&apos;Malley"')
combo_name_url.pop('"John  Gray"')
combo_name_url.pop('"Gray John"')
combo_name_url.pop('"Euripides; Paul Roche (Transl.)"')
combo_name_url.pop('"James J. O&apos;Donnell"')
combo_name_url.pop('"Wild at Heart Field Manual: A Personal Guide to Discover the Secret of Your Masculine Soul"')
combo_name_url.pop('"John D&apos;Emilio"')
combo_name_url.pop('"Into Thin Air: A Personal Account of the Mt. Everest Disaster"')
combo_name_url.pop('"Mtetwa &quot;Tet&quot; Ramdoo"')
combo_name_url.pop('"Gillen D&apos;Arcy Wood"')

'https://www.goodreads.com/author/show/182153.Gillen_D_Arcy_Wood]'

In [ ]:
#Updating our author_urls list again:
author_urls = list(combo_name_url.values())

In [ ]:
#Initialising our empty list which will contain information about our authors:
about_author = list()

#Obtaining information on the authors:
c = 1

for au in author_urls:
  if au is None:
    about_author += [None]
    print (c,au,'empty')
    c += 1
    continue

  url = au
  print (c, '--> ',url)

  page = urlopen(url)
  html_bytes = page.read()
  html = html_bytes.decode('utf-8')

  url_split = au.split('/')

  author_id = url_split[len(url_split)-1].split('.')[0]

  author_soup = BeautifulSoup(html,'html.parser')
  about_section = author_soup.find_all("span", id = f"freeTextauthor{author_id}")

  if len(about_section) == 0:
    about_author += ['']
  else:
    for i in range(len(about_section)):
      about_author += [about_section[i].getText()]
  c += 1
  if c % 50 == 0:
    time.sleep(10)

1 -->  https://www.goodreads.com/author/show/1077326.J_K_Rowling]
2 -->  https://www.goodreads.com/author/show/2927.Mary_GrandPr_]
3 -->  https://www.goodreads.com/author/show/3.W_Frederick_Zimmerman]
4 -->  https://www.goodreads.com/author/show/4.Douglas_Adams
5 -->  https://www.goodreads.com/author/show/951394.Jamie_Flinchbaugh
6 -->  https://www.goodreads.com/author/show/6910651.Andy_Carlino
7 -->  https://www.goodreads.com/author/show/2740859.Dennis_Pawley]
8 -->  https://www.goodreads.com/author/show/10917.Stephen_Fry
9 -->  https://www.goodreads.com/author/show/7.Bill_Bryson]
10 -->  https://www.goodreads.com/author/show/656983.J_R_R_Tolkien
11 -->  https://www.goodreads.com/author/show/42155.Francis_Ledoux
12 -->  https://www.goodreads.com/author/show/2818175.Maria_Skibniewska]
13 -->  https://www.goodreads.com/author/show/9545.Alan_Lee]
14 -->  https://www.goodreads.com/author/show/5448409.Chris_Smith
15 -->  https://www.goodreads.com/author/show/5325360.Christopher_Lee
16 --> 

In [ ]:
#Obtaining our updated author names (without authors that were removed):
author_names = list(combo_name_url.keys())

In [ ]:
#Checking our list lengths:
print (len(first_names))
print (len(last_names))
print (len(author_urls))
print (len(about_author))

1910
1910
1910
1910


In [ ]:
#Pulling the first and last names from the combined strings in author_names:
first_names = [fn.split(' ')[0].translate({ord('"'):None}) for fn in author_names]

last_names = list()
for ln in author_names:
  if len(ln.split(' ',1))>=2:
    ln = ln.split(' ',1)[1].translate({ord('"'):None})
    last_names += [ln]
  else:
    last_names += [None]

In [ ]:
#Placing our author information in a dict:
author_info = dict()
author_id = range(1,len(author_names)+1)
author_info['First Name'] = first_names
author_info['Last Name'] = last_names
author_info['About Author'] = about_author

In [ ]:
#Putting the authors information in a data-frame:
author_df = pd.DataFrame(author_info, index = author_id)
author_df

,First Name,Last Name,About Author
1,J.K.,Rowling,See also: Robert GalbraithAlthough she writes ...
2,Mary,GrandPré,Mary GrandPré is an American illustrator best ...
3,W.,Frederick Zimmerman,I have worked at the intersection of computers...
4,Douglas,Adams,Librarian Note: There is more than one author ...
5,Jamie,Flinchbaugh,"As Founder of JFlinch, Jamie Flinchbaugh has h..."
...,...,...,...
1906,Jane,Smiley,Jane Smiley is a Pulitzer Prize-winning Americ...
1907,Hugh,Laurie,"British comedian Hugh Laurie, OBE, could have ..."
1908,Patricia,Ingham,Patricia Ingham is senior research fellow and ...
1909,Connie,Ann Kirk,


In [ ]:
#Saving our author information to a csv:
author_df.to_csv('author_df.csv')

# **Book Content**

In [ ]:
#Creating an empty books data list:
books_data = list()

#Iterating over our books:
for bb_ in book_info:
  bb_book = bb_[0]
  gs = bb_[1]

  try:
    book = json.loads(bb_book.strip())

    title = book.get("name", "")
    authors = [a.get("name", "") for a in book.get("author", [])]
    rating_info = book.get("aggregateRating", {})
    rating = rating_info.get("ratingValue", None)
    rating_count = rating_info.get("ratingCount", None)
    review_count = rating_info.get("reviewCount", None)
    pages = book.get("numberOfPages", None)
    book_format = book.get("bookFormat", "")

    #Split awards into a list (handling string or already list format)
    awards = book.get("awards", [])
    if isinstance(awards, str):
        awards = [a.strip() for a in awards.split(",") if a.strip()]

    books_data.append({
        "Title": title,
        "Author(s)": authors,
        "Rating": rating,
        "Rating_count": rating_count,
        "Review_count": review_count,
        "No. of Pages": pages,
        "Format": book_format,
        "Awards": awards,
        "Genres":gs
    })

  except json.JSONDecodeError as e:
    print("Invalid JSON line:", line)
    continue

In [ ]:
#Converting our data into a data-frame:
book_ids = range(1,len(book_info)+1)
books_df = pd.DataFrame(books_data,index = book_ids)

#Combining the author, awards and genres seperated by commas:
books_df["Author(s)"] = books_df["Author(s)"].apply(lambda x: ", ".join(x))
books_df["Awards"] = books_df["Awards"].apply(lambda x: ", ".join(x) if isinstance(x, list) else "")
books_df["Genres"] = books_df["Genres"].apply(lambda x: ", ".join(x))

#Displaying our data-frame:
books_df

,Title,Author(s),Rating,Rating_count,Review_count,No. of Pages,Format,Awards,Genres
1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.58,3540997.0,66336.0,652.0,Paperback,"Locus Award Best Young Adult Novel (2006), Aud...","Fantasy, Fiction, Young Adult, Magic, Audioboo..."
2,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling, Mary GrandPré",4.50,3666223.0,73250.0,912.0,Paperback,Bram Stoker Award Works for Young Readers (200...,"Fantasy, Young Adult, Fiction, Magic, Audioboo..."
3,Harry Potter and the Sorcerer&apos;s Stone (Ha...,J.K. Rowling,4.47,10984901.0,182647.0,309.0,Hardcover,Mythopoeic Fantasy Award Children&apos;s Liter...,"Fantasy, Fiction, Young Adult, Magic, Children..."
4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.43,4316879.0,91245.0,352.0,Hardcover,Mythopoeic Fantasy Award Children&apos;s Liter...,"Fantasy, Fiction, Young Adult, Magic, Children..."
5,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPré",4.58,4639665.0,96500.0,435.0,Mass Market Paperback,Bram Stoker Award Best Work for Young Readers ...,"Fantasy, Fiction, Young Adult, Magic, Children..."
...,...,...,...,...,...,...,...,...,...
1927,Collapse: How Societies Choose to Fail or Survive,Jared Diamond,3.93,73697.0,3907.0,592.0,Hardcover,"Royal Society Science Book Prize (2006), Calif...","History, Nonfiction, Science, Anthropology, So..."
1928,Colapso: Por qué unas sociedades perduran y ot...,Jared Diamond,3.93,73697.0,3907.0,752.0,Hardcover,"Royal Society Science Book Prize (2006), Calif...","History, Nonfiction, Science, Anthropology, So..."
1929,J. K. Rowling: A Biography (Unauthorized Edition),Connie Ann Kirk,3.46,114.0,12.0,155.0,Hardcover,,"Biography, Nonfiction"
1930,The J.K. Rowling Encyclopedia,Connie Ann Kirk,4.42,24.0,NaN,374.0,Hardcover,,


In [ ]:
#Saving books to a csv file:
books_df.to_csv('book_info.csv')

# **Users**
Creating a data-frame for Fable users:

In [ ]:
#Reading in our data:
reviewer_data_df = pd.read_csv('/content/reviewers.csv',index_col = 0)
display(reviewer_data_df)

,Username
1,Jayson
2,Jayson
3,Whitney Atkinson
4,Nicole
5,Tharindu Dissanayake
...,...
33198,Olivia Cornwell
33199,Nicole
33200,Cheri Herald
33201,Susy


In [ ]:
#Removing duplicate values from our reviewer df:
reviewer_data_df = reviewer_data_df.drop_duplicates(ignore_index = True)
reviewer_data_df = reviewer_data_df.rename(index = dict(zip(list(range(0,len(reviewer_data_df))), list(range(1,len(reviewer_data_df)+1)))))
display(reviewer_data_df)

,Username
1,Jayson
2,Whitney Atkinson
3,Nicole
4,Tharindu Dissanayake
5,Chelsea Humphrey
...,...
11641,Arya
11642,Olivia Cornwell
11643,Cheri Herald
11644,Susy


In [ ]:
#Saving our new reviewer_df:
reviewer_data_df.to_csv('reviewers_new.csv')

In [ ]:
#Reading in our external names file:
names_df = pd.read_csv('/content/NationalNames.csv')
display(names_df)

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746
...,...,...,...,...,...
1825428,1825429,Zykeem,2014,M,5
1825429,1825430,Zymeer,2014,M,5
1825430,1825431,Zymiere,2014,M,5
1825431,1825432,Zyran,2014,M,5


In [ ]:
#Creating a function that generates random dates in a range:
def random_date(start_date, end_date):
  time_between_dates = end_date - start_date
  days_between_dates = time_between_dates.days
  random_number_of_days = random.randrange(days_between_dates)
  random_date = start_date + datetime.timedelta(days=random_number_of_days)
  return random_date

In [ ]:
import datetime

#Collecting random first and last names:
first_names_rd = random.sample(list(names_df['Name']),1000)
last_names_rd = random.sample(list(names_df['Name']),1000)

#Combining first and last names:
full_names = list(zip(first_names_rd,last_names_rd))
full_names = [str(fn) for fn in full_names]

#Creating user names:
user_names = list()
for it in range(1000):
  user_names += [first_names_rd[it] + random.choice(['_','.',''])+ last_names_rd[it]]

#Creating random dates of birth:
dobs = list()
start_date = datetime.date(1900, 1, 1)
end_date = datetime.date(2025, 1, 1)

for it_dob in range(1000):
  dobs += [random_date(start_date,end_date)]

#Creating a list of email extensions:
email_enders = ['@gmail.com','@hotmail.com','@yahoo.com']

#Creating emails:
email_addresses = list()
for fn in first_names_rd:
  ender = random.choice(email_enders)
  email_addresses += [fn + ender]


In [ ]:
#Creating a users data-frame:
user_id = list(range(1,1000+1))
users_dict = {'Username':user_names, 'First Name':first_names_rd, 'Last Name':last_names_rd,'EmailAddress':email_addresses,
              'DOB':dobs,'AddressID':address_id}
users_df = pd.DataFrame(users_dict, index = user_id)
display(users_df)

,Username,First Name,Last Name,EmailAddress,DOB
1,Zykia.Brandt,Zykia,Brandt,Zykia@yahoo.com,1979-02-11
2,Arlena_Camille,Arlena,Camille,Arlena@gmail.com,1913-06-21
3,Nevaehlynn_Skyasia,Nevaehlynn,Skyasia,Nevaehlynn@hotmail.com,2014-08-07
4,Adreana_Blaze,Adreana,Blaze,Adreana@gmail.com,1973-03-01
5,DestaniMelodi,Destani,Melodi,Destani@yahoo.com,1931-02-06
...,...,...,...,...,...
996,GabrielaAlexander,Gabriela,Alexander,Gabriela@yahoo.com,1938-02-13
997,Acie.Taiylor,Acie,Taiylor,Acie@hotmail.com,1927-04-17
998,Patsy.Ara,Patsy,Ara,Patsy@yahoo.com,2017-12-19
999,Kacee_Effrey,Kacee,Effrey,Kacee@yahoo.com,2013-06-05


In [ ]:
reviewer_user_names = list(reviewer_data_df['Username'])
reviewer_emails = list()

#Obtaining the reviewer first and last names:
first_name_reviewers = [fn.split(' ')[0] if len(fn.split(' '))>1 else fn for fn in reviewer_user_names]
last_name_reviewers = [fn.split(' ')[1] if len(fn.split(' '))>1 else '' for fn in reviewer_user_names]

#Creating reviewer emails:
for fn in first_name_reviewers:
  ender = random.choice(email_enders)
  reviewer_emails += [fn + ender]

#Obtaining the reviewer birthdays:
reviewer_dobs = list()
for it_dob in range(len(reviewer_data_df)):
  reviewer_dobs += [random_date(start_date,end_date)]

user_id = list(range(1,1000+len(reviewer_data_df)+1))

#Combining together our data:
users_usernames = user_names + reviewer_user_names
users_emails = email_addresses + reviewer_emails
users_firstnames = first_names_rd + first_name_reviewers
users_lastnames = last_names_rd + last_name_reviewers
users_dobs = dobs + reviewer_dobs

In [ ]:
users_dict = {'Username':users_usernames, 'First Name':users_firstnames , 'Last Name':users_lastnames,'EmailAddress':users_emails,
              'DOB':users_dobs,'AddressID':address_id}
users_df = pd.DataFrame(users_dict, index = user_id)
display(users_df)

,Username,First Name,Last Name,EmailAddress,DOB,AddressID
1,OnniePesach,Onnie,Pesach,Onnie@yahoo.com,2022-06-11,1
2,Heymi.Berenice,Heymi,Berenice,Heymi@yahoo.com,1943-01-29,2
3,Terran_Yolanda,Terran,Yolanda,Terran@gmail.com,2006-09-05,3
4,Awilda.Hikari,Awilda,Hikari,Awilda@hotmail.com,1967-11-27,4
5,Paulette.Vance,Paulette,Vance,Paulette@hotmail.com,1926-09-21,5
...,...,...,...,...,...,...
12641,Arya,Arya,Arya,Arya@gmail.com,1963-01-22,12641
12642,Olivia Cornwell,Olivia,Cornwell,Olivia@gmail.com,2009-09-11,12642
12643,Cheri Herald,Cheri,Herald,Cheri@hotmail.com,2023-04-07,12643
12644,Susy,Susy,Susy,Susy@yahoo.com,1940-01-21,12644


In [ ]:
#Saving to a csv:
users_df.to_csv('users_info.csv')

# **User Addresses.**

In [ ]:
#Reading in our addresses file:
addresses = pd.read_csv('/content/Real_Estate_Sales_2001-2020_GL.csv')

#Observing what is in our csv file:
display(addresses)

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Years until sold
0,2020348,2020,9/13/2021,Ansonia,230 WAKELEE AVE,150500,325000.0,0.463000,Commercial,Nan,1
1,20002,2020,10/2/2020,Ashford,390 TURNPIKE RD,253000,430000.0,0.588300,Residential,Single Family,0
2,200212,2020,3/9/2021,Avon,5 CHESTNUT DRIVE,130400,179900.0,0.724800,Residential,Condo,1
3,200243,2020,4/13/2021,Avon,111 NORTHINGTON DRIVE,619290,890000.0,0.695800,Residential,Single Family,1
4,200377,2020,7/2/2021,Avon,70 FAR HILLS DRIVE,862330,1447500.0,0.595700,Residential,Single Family,1
...,...,...,...,...,...,...,...,...,...,...,...
997208,190272,2019,6/24/2020,New London,4 BISHOP CT,60410,53100.0,1.137665,Single Family,Single Family,1
997209,190284,2019,11/27/2019,Waterbury,126 PERKINS AVE,68280,76000.0,0.898400,Single Family,Single Family,0
997210,190129,2019,4/27/2020,Windsor Locks,19 HATHAWAY ST,121450,210000.0,0.578300,Single Family,Single Family,1
997211,190504,2019,6/3/2020,Middletown,8 BYSTREK DR,203360,280000.0,0.726300,Single Family,Single Family,1


In [ ]:
#Randomly generating user addresses from the csv file:
user_town = random.sample(list(addresses['Town']),1000+len(reviewer_data_df))
user_address = random.sample(list(addresses['Address']),1000+len(reviewer_data_df))
country = ['USA']*len(user_address)

address_id = list(range(1,1000+len(reviewer_data_df)+1))
address_data = {'UserID':user_id,'Town':user_town,'Address':user_address}

address_info = pd.DataFrame(address_data,address_id)
display(address_info)

,UserID,Town,Address
1,1,New Haven,LAKESIDE DR
2,2,Simsbury,5 SAXON WOODS
3,3,Thomaston,198 BEACON RD
4,4,Newington,102 WILD HORSE CT
5,5,Middletown,325 MAIN ST S UT 4
...,...,...,...
12641,12641,New Haven,8 HIGHLAND DR
12642,12642,East Hartford,125 SONGBIRD LN
12643,12643,Naugatuck,33 SUNSET RIDGE
12644,12644,Trumbull,1 ORWICH CT


In [ ]:
#Saving to a csv:
address_info.to_csv('addresses.csv')